#Semantic Segmentation of Aerial Images
###Road Training

####Mounting Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')
base_path = "gdrive/My\ Drive/MapSegClean/"
%cd gdrive/My\ Drive/MapSegClean/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/
/content/gdrive/My Drive/MapSegClean


####Importing Libraries

In [0]:
!nvidia-smi

Wed Mar 11 05:36:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import cv2
import numpy as np
import h5py

In [0]:
import tensorflow as tf
from keras import backend as K

Using TensorFlow backend.


In [0]:
from keras.models import Model, load_model
import tensorflow as tf
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras import optimizers
from keras.layers import BatchNormalization
from tensorflow.keras.metrics import MeanIoU
import keras

from skimage import color
import pickle

####Read Data From h5 files

In [0]:
h5f = h5py.File('Cpy_clean_roadlabel.h5','r')
roadlabel = h5f['clean_roadlabel'][:]
h5f.close()

h5f = h5py.File('Cpy_clean_road.h5','r')
road = h5f['clean_road'][:]
h5f.close()

In [0]:
#for i in range(road.shape[0]):
#  cv2.imwrite("Road/" + str(i) + ".png", road[i])

#for i in range(roadlabel.shape[0]):
#  cv2.imwrite("RoadLabel/" + str(i) + ".png", roadlabel[i])
pip install -U segmentation-models

####Loss Function

In [0]:
def iou_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
  union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
  iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  return iou

def dice_coef(y_true, y_pred, smooth = 1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def soft_dice_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)


####Preprocecssing Data

In [0]:
print(roadlabel.shape)

roadlabelgray = []

for i in range(roadlabel.shape[0]):
  roadlabelgray.append(cv2.cvtColor(roadlabel[i], cv2.COLOR_BGR2GRAY))

roadlabelgray = np.asarray(roadlabelgray)

roadlabel = np.expand_dims(roadlabelgray, -1)

del roadlabelgray

print(roadlabel.shape)

(22911, 256, 256, 3)
(22911, 256, 256, 1)


In [0]:
seed = 56
IMAGE_HEIGHT = IMAGE_WIDTH = 256
NUM_CHANNELS = 3

####Model

In [0]:
from segmentation_models import PSPNet
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score

Segmentation Models: using `keras` framework.


In [0]:
model = PSPNet('efficientnetb7',input_shape=(240, 240, 3), encoder_weights=None, classes=1, psp_use_batchnorm=True, psp_pooling_type='max', activation='sigmoid')

#model = FPN(input_shape=(256, 256, 3), classes = 1, pyramid_use_batchnorm=True, activation='softmax', encoder_weights=None)
#model = Unet(input_shape=(256, 256, 3), weights=None, activation='elu')
model.summary()












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 240, 240, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 120, 120, 64) 1728        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 120, 120, 64) 256         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 120, 120, 64) 0        

####Hyperparameters


In [0]:
LEARNING_RATE = 0.0001
BATCH_SIZE = 32

####Path to save Model

In [0]:
model_path = "Models/Final_PSPNet_efficientnetb7_road_weights.h5"

In [0]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                          road:  4.2 GiB
                         Model:  2.0 KiB
                        Conv2D:  2.0 KiB
            BatchNormalization:  2.0 KiB
                  MaxPooling2D:  1.4 KiB
                       Dropout:  1.0 KiB
                        Lambda:  1.0 KiB
               Conv2DTranspose:  1.0 KiB
                       MeanIoU:  1.0 KiB
                           _i8:  615.0 B


####Callbacks

In [0]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
tbc=TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
https://c6abf5e1.ngrok.io


In [0]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from datetime import datetime

checkpointer = ModelCheckpoint(model_path,
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

In [0]:
earlystopper = EarlyStopping(monitor = 'val_loss', 
                             min_delta = 0, 
                             patience = 7,
                             mode='auto',
                            restore_best_weights = True)

In [0]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4)

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1335: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


####Compile Model

In [0]:
opt = keras.optimizers.adam(LEARNING_RATE)
model.compile(
      optimizer=opt,
      loss=soft_dice_loss,
      metrics=[iou_coef])   

In [0]:
history = model.fit(road[:, 8:256-8, 8:256-8],
                    (roadlabel[:, 8:256-8, 8:256-8].astype(np.uint8)/255).astype(np.uint8),
                    validation_split = 0.1,
                    epochs=100,
                    batch_size = 16,
                    callbacks = [checkpointer, earlystopper, lr_reducer, TensorBoardColabCallback(tbc)]
                       )

with open("History/PSPNet_efficientnetb7_Road_History.pickle", 'wb') as f:
    pickle.dump(history, f)



Train on 20619 samples, validate on 2292 samples


Epoch 1/100
20619/20619 [==============================] - 689s 33ms/step - loss: 0.6211 - iou_coef: 0.2067 - val_loss: 0.4414 - val_iou_coef: 0.3319

Epoch 00001: val_loss improved from inf to 0.44144, saving model to Models/Final_PSPNet_efficientnetb7_road_weights.h5

Epoch 2/100
20619/20619 [==============================] - 670s 32ms/step - loss: 0.4130 - iou_coef: 0.4105 - val_loss: 0.3747 - val_iou_coef: 0.4723

Epoch 00002: val_loss improved from 0.44144 to 0.37473, saving model to Models/Final_PSPNet_efficientnetb7_road_weights.h5
Epoch 3/100
20619/20619 [==============================] - 671s 33ms/step - loss: 0.3652 - iou_coef: 0.4729 - val_loss: 0.3431 - val_iou_coef: 0.5086

Epoch 00003: val_loss improved from 0.37473 to 0.34314, saving model to Models/Final_PSPNet_efficientnetb7_road_weights.h5
Epoch 4/100
20619/20619 [==============================] - 671s 33ms/step - loss: 0.3439 - iou_coef: 0.5080 - val_loss: 0.3272 -

In [0]:
model.load_weights(model_path)

history = model.fit(road[:, 8:256-8, 8:256-8],
                    (roadlabel[:, 8:256-8, 8:256-8].astype(np.uint8)/255).astype(np.uint8),
                    validation_split = 0.1,
                    epochs=40,
                    batch_size = 16,
                    callbacks = [checkpointer, earlystopper, lr_reducer, TensorBoardColabCallback(tbc)]
                       )

with open("History/PSPNet_efficientnetb7_Road_History_1.pickle", 'wb') as f:
    pickle.dump(history, f



Train on 20619 samples, validate on 2292 samples


Epoch 1/40
20619/20619 [==============================] - 683s 33ms/step - loss: 0.2873 - iou_coef: 0.5854 - val_loss: 0.2859 - val_iou_coef: 0.5952

Epoch 00001: val_loss improved from inf to 0.28591, saving model to Models/Final_PSPNet_efficientnetb7_road_weights.h5

Epoch 2/40
20619/20619 [==============================] - 661s 32ms/step - loss: 0.2836 - iou_coef: 0.5890 - val_loss: 0.2877 - val_iou_coef: 0.5924

Epoch 00002: val_loss did not improve from 0.28591
Epoch 3/40
20619/20619 [==============================] - 661s 32ms/step - loss: 0.2802 - iou_coef: 0.5926 - val_loss: 0.2860 - val_iou_coef: 0.5971

Epoch 00003: val_loss did not improve from 0.28591
Epoch 4/40
20619/20619 [==============================] - 661s 32ms/step - loss: 0.2784 - iou_coef: 0.5972 - val_loss: 0.2839 - val_iou_coef: 0.5921

Epoch 00004: val_loss improved from 0.28591 to 0.28394, saving model to Models/Final_PSPNet_efficientnetb7_road_weights.h5
Ep

In [0]:
opt = keras.optimizers.adam(10e-6)
model.compile(
      optimizer=opt,
      loss=soft_dice_loss,
      metrics=[iou_coef])   

In [0]:
model.load_weights(model_path)

history = model.fit(road[:, 8:256-8, 8:256-8],
                    (roadlabel[:, 8:256-8, 8:256-8].astype(np.uint8)/255).astype(np.uint8),
                    validation_split = 0.1,
                    epochs=40,
                    batch_size = 16,
                    callbacks = [checkpointer, earlystopper, lr_reducer, TensorBoardColabCallback(tbc)]
                       )

with open("History/PSPNet_efficientnetb7_Road_History_2.pickle", 'wb') as f:
    pickle.dump(history, f)

Train on 20619 samples, validate on 2292 samples
Epoch 1/40
20619/20619 [==============================] - 1842s 89ms/step - loss: 0.2376 - iou_coef: 0.6507 - val_loss: 0.2646 - val_iou_coef: 0.6179

Epoch 00001: val_loss improved from inf to 0.26459, saving model to Models/Final_PSPNet_efficientnetb7_road_weights.h5

Epoch 2/40
20619/20619 [==============================] - 1852s 90ms/step - loss: 0.2372 - iou_coef: 0.6526 - val_loss: 0.2644 - val_iou_coef: 0.6190

Epoch 00002: val_loss improved from 0.26459 to 0.26443, saving model to Models/Final_PSPNet_efficientnetb7_road_weights.h5
Epoch 3/40
20619/20619 [==============================] - 1850s 90ms/step - loss: 0.2368 - iou_coef: 0.6521 - val_loss: 0.2643 - val_iou_coef: 0.6193

Epoch 00003: val_loss improved from 0.26443 to 0.26426, saving model to Models/Final_PSPNet_efficientnetb7_road_weights.h5
Epoch 4/40
20619/20619 [==============================] - 1852s 90ms/step - loss: 0.2363 - iou_coef: 0.6539 - val_loss: 0.2639 - val

In [0]:
opt = keras.optimizers.adam(10e-7)
model.compile(
      optimizer=opt,
      loss=soft_dice_loss,
      metrics=[iou_coef])   

In [0]:
model.load_weights(model_path)

history = model.fit(road[:, 8:256-8, 8:256-8],
                    (roadlabel[:, 8:256-8, 8:256-8].astype(np.uint8)/255).astype(np.uint8),
                    validation_split = 0.1,
                    epochs=40,
                    batch_size = 16,
                    callbacks = [checkpointer, earlystopper, lr_reducer, TensorBoardColabCallback(tbc)]
                       )

with open("History/PSPNet_efficientnetb7_Road_History_2.pickle", 'wb') as f:
    pickle.dump(history, f)



Train on 20619 samples, validate on 2292 samples


Epoch 1/40
20619/20619 [==============================] - 1803s 87ms/step - loss: 0.2350 - iou_coef: 0.6548 - val_loss: 0.2638 - val_iou_coef: 0.6193

Epoch 00001: val_loss improved from inf to 0.26382, saving model to Models/Final_PSPNet_efficientnetb7_road_weights.h5

Epoch 2/40
20619/20619 [==============================] - 1779s 86ms/step - loss: 0.2344 - iou_coef: 0.6563 - val_loss: 0.2638 - val_iou_coef: 0.6197

Epoch 00002: val_loss did not improve from 0.26382
Epoch 3/40
20619/20619 [==============================] - 1773s 86ms/step - loss: 0.2349 - iou_coef: 0.6544 - val_loss: 0.2639 - val_iou_coef: 0.6193

Epoch 00003: val_loss did not improve from 0.26382
Epoch 4/40
20619/20619 [==============================] - 1773s 86ms/step - loss: 0.2343 - iou_coef: 0.6559 - val_loss: 0.2640 - val_iou_coef: 0.6191

Epoch 00004: val_loss did not improve from 0.26382
Epoch 5/40
20619/20619 [==============================] - 1778s 86ms/s

In [0]:
#import pickle
#with open("History/Road_History.pickle", 'rb') as f:
#    history = pickle.load(f)

In [0]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['iou_coef'])
plt.plot(history.history['val_iou_coef'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()